Projeto 2 - Ciência dos dados

Integrantes: 
Luca Guazzelli 
Pedro Henrique Vidal 
Rafael Agnelo

A base de dados é uma base de microdados do ENEM em relação a dados do ENEM por escolas (públicas e privadas) no Brasil. Em relação à isso, faz-se a seguinte pergunta temática:

A partir de uma série de features (Localidade, Taxa de abandono, Porte da escola , Média das notas em matemática, Tipo da dependência administrativa, Número de participantes do enem que cumprem os requisitos de participação e INSE), qual é o perfil médio de uma escola que performa mal, isto é, de forma insatisfatória em Matemática no ENEM (Média inferior à 500)?



In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [9]:
dados = pd.read_csv('microdados_enem_por_escola\DADOS\MICRODADOS_ENEM_ESCOLA.csv',encoding='latin1', sep=";")

C:\Users\lucah\AppData\Local\Temp\ipykernel_96288\2310550405.py:1: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  dados = pd.read_csv('microdados_enem_por_escola\DADOS\MICRODADOS_ENEM_ESCOLA.csv',encoding='latin1', sep=";")


In [10]:
dados

,NU_ANO,CO_UF_ESCOLA,SG_UF_ESCOLA,CO_MUNICIPIO_ESCOLA,NO_MUNICIPIO_ESCOLA,CO_ESCOLA_EDUCACENSO,NO_ESCOLA_EDUCACENSO,TP_DEPENDENCIA_ADM_ESCOLA,TP_LOCALIZACAO_ESCOLA,NU_MATRICULAS,...,NU_MEDIA_RED,NU_MEDIA_OBJ,NU_MEDIA_TOT,INSE,PC_FORMACAO_DOCENTE,NU_TAXA_PERMANENCIA,NU_TAXA_APROVACAO,NU_TAXA_REPROVACAO,NU_TAXA_ABANDONO,PORTE_ESCOLA
0,2007,11,RO,1100205,Porto Velho,11000058,CENTRO DE ENSINO CLASSE A,4,1,144,...,NaN,NaN,69.03,NaN,NaN,NaN,91.9,8.1,0.0,Maior que 90 alunos
1,2006,11,RO,1100205,Porto Velho,11000058,CENTRO DE ENSINO CLASSE A,4,1,184,...,NaN,NaN,57.82,NaN,NaN,NaN,NaN,NaN,NaN,Maior que 90 alunos
2,2005,11,RO,1100205,Porto Velho,11000058,CENTRO DE ENSINO CLASSE A,4,1,220,...,NaN,NaN,64.83,NaN,NaN,NaN,86.5,12.4,1.1,Maior que 90 alunos
3,2008,11,RO,1100205,Porto Velho,11000058,CENTRO DE ENSINO CLASSE A,4,1,186,...,72.16,60.02,NaN,NaN,NaN,NaN,90.3,9.7,0.0,Maior que 90 alunos
4,2007,11,RO,1100205,Porto Velho,11000171,CENTRO EDUCACIONAL MOJUCA,4,1,19,...,NaN,NaN,58.84,NaN,NaN,NaN,74.2,21.0,4.8,De 1 a 30 alunos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172300,2013,53,DF,5300108,Brasília,53068181,CED 310 DE SANTA MARIA,2,1,213,...,503.86,NaN,NaN,NaN,91.8,NaN,83.1,9.1,7.8,Maior que 90 alunos
172301,2014,53,DF,5300108,Brasília,53068181,CED 310 DE SANTA MARIA,2,1,138,...,470.20,NaN,NaN,NaN,72.0,82.00,81.2,17.5,1.3,Maior que 90 alunos
172302,2015,53,DF,5300108,Brasília,53068181,CED 310 DE SANTA MARIA,2,1,198,...,517.24,NaN,NaN,Grupo 3,83.3,76.32,78.9,10.1,11.0,Maior que 90 alunos
172303,2014,53,DF,5300108,Brasília,53082001,COL IMPACTO,4,1,12,...,482.00,NaN,NaN,NaN,69.2,NaN,91.4,8.6,0.0,De 1 a 30 alunos
